In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import gspread
import pandas as pd

In [6]:
header = {
'scheme':'https',
'Accept':'application/json, text/plain, */*',
'Accept-Language':'en-US,en;q=0.8',
'Referer':'https://www.bseindia.com/',
'Sec-Ch-Ua':'"Not_A Brand";v="8", "Chromium";v="120", "Brave";v="120"',
'Sec-Ch-Ua-Mobile': '?0',
'Sec-Ch-Ua-Platform':'"Windows"',
'Sec-Fetch-Mode':'cors',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
chrome_options = Options()
chrome_options.add_argument('--headless')
for key, value in header.items():
    chrome_options.add_argument(f"{key}={value}")
driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.bseindia.com/corporates/Forth_Results.html'
driver.get(url)
source = driver.page_source

In [7]:
gc = gspread.service_account(filename=r'C:\Users\Ashish Kumar Pal\OneDrive\Desktop\Python\Exchange Related task\NSE_Projtects\creds.json')
spreadsheet_name = 'QuarterlyResultsTracker'
sheet_name = 'Result calendar'
sh = gc.open(spreadsheet_name).worksheet(sheet_name)

In [8]:
soup = bs(source,'html.parser')
Table = soup.find('table',{'class':'ng-scope'})
table_content = []
rows = Table.find_all('tr')
for row in rows:
    cells = row.find_all(['td', 'th'])
    row_content = [cell.get_text(strip=True) for cell in cells]
    table_content.append(row_content)

In [9]:
Column = table_content[1]
MainDf = table_content[2:]
df = pd.DataFrame(MainDf,columns = Column)

In [10]:
existing_data_range = sh.range('A2:F' + str(sh.row_count))
for cell in existing_data_range:
    cell.value = ''

sh.update_cells(existing_data_range)

{'spreadsheetId': '1RVcXIyD754djShGJR3_Xd78BQ2hLpzC8plgoc9qnFRE',
 'updatedRange': "'Result calendar'!A2:F2685",
 'updatedRows': 2684,
 'updatedColumns': 6,
 'updatedCells': 16104}

In [11]:
values_lists = []
Data_Dict = df.to_dict(orient='records')
for data in Data_Dict:
    values_lists.append([int(data['Security Code']), str(data['Security Name']), str(data['Result Date'])])
sh.append_rows(values_lists)

{'spreadsheetId': '1RVcXIyD754djShGJR3_Xd78BQ2hLpzC8plgoc9qnFRE',
 'tableRange': "'Result calendar'!A1:C1",
 'updates': {'spreadsheetId': '1RVcXIyD754djShGJR3_Xd78BQ2hLpzC8plgoc9qnFRE',
  'updatedRange': "'Result calendar'!A2:C1629",
  'updatedRows': 1628,
  'updatedColumns': 3,
  'updatedCells': 4884}}